In [2]:
"""
Automating Pre-processing with Pipelines in scikit-learn (Numeric-focused)

Dataset: retail_sales_week4.csv
Target : target_sales

What this program demonstrates:
1) Load dataset + apply simple validity rules (0 income -> missing, negative marketing spend -> missing)
2) Train/Test split FIRST (avoid leakage)
3) Automatically detect skewed numeric features from TRAIN split only
4) Build a ColumnTransformer that:
   - Applies (Impute -> PowerTransform -> Scale) to skewed numeric columns
   - Applies (Impute -> Scale) to non-skewed numeric columns
5) Wrap everything inside a single Pipeline with a model
6) Evaluate baseline vs automated pipeline
7) Show how to reuse the same pipeline for future data (predict)

Why PowerTransformer?
- Works for many skew patterns
- Doesn't require strictly positive values (Yeo-Johnson)
- Great for "automated" preprocessing across many columns
"""
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import Ridge


In [5]:
# -----------------------------
# 1) Load dataset
# -----------------------------
df = pd.read_csv("D:/datasets/dpp/retail_sales_week4.csv")

TARGET = "target_sales"
ID_COLS = ["customer_id", "store_id"]



In [6]:
# -----------------------------
# 2) Validity cleaning (safe business rules; OK before split)
# -----------------------------
df_clean = df.copy()
df_clean.loc[df_clean["annual_income"] <= 0, "annual_income"] = np.nan
df_clean.loc[df_clean["marketing_spend"] < 0, "marketing_spend"] = np.nan



In [7]:
# -----------------------------
# 3) Select numeric features
# -----------------------------
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [c for c in numeric_cols if c not in ID_COLS + [TARGET]]

X = df_clean[numeric_features]
y = df_clean[TARGET]

print("Rows:", len(df_clean))
print("Numeric features:", numeric_features)


Rows: 1000
Numeric features: ['age', 'annual_income', 'monthly_spend', 'discount_percentage', 'purchase_frequency', 'avg_basket_value', 'avg_transaction_value', 'days_since_last_purchase', 'online_visits', 'total_returns', 'loyalty_score', 'marketing_spend', 'energy_consumption_store']


In [8]:
# -----------------------------
# 4) Split train/test FIRST (avoid leakage)
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [9]:
# -----------------------------
# 5) Automatically detect skewed columns (TRAIN only)
# -----------------------------
# Rule: |skew| >= 1.0 => "skewed"
SKEW_THRESHOLD = 1.0

train_skew = X_train.skew(numeric_only=True)
skewed_cols = train_skew[train_skew.abs() >= SKEW_THRESHOLD].index.tolist()
non_skewed_cols = [c for c in numeric_features if c not in skewed_cols]

print("\nSkewness (train) top 10 by |skew|:")
print(train_skew.reindex(train_skew.abs().sort_values(ascending=False).index).head(10).round(3))

print("\nSkewed cols (|skew| >= 1.0):", skewed_cols)
print("Non-skewed cols:", non_skewed_cols)



Skewness (train) top 10 by |skew|:
total_returns               9.109
annual_income               8.755
monthly_spend               8.162
avg_transaction_value       7.442
avg_basket_value            7.353
marketing_spend             6.925
energy_consumption_store    5.920
days_since_last_purchase    4.632
discount_percentage         0.902
purchase_frequency          0.835
dtype: float64

Skewed cols (|skew| >= 1.0): ['annual_income', 'monthly_spend', 'avg_basket_value', 'avg_transaction_value', 'days_since_last_purchase', 'total_returns', 'marketing_spend', 'energy_consumption_store']
Non-skewed cols: ['age', 'discount_percentage', 'purchase_frequency', 'online_visits', 'loyalty_score']


In [10]:
# -----------------------------
# 6) Build preprocessing pipelines
# -----------------------------
# Skewed numeric pipeline: Impute -> PowerTransform -> Scale
skewed_num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("power", PowerTransformer(method="yeo-johnson", standardize=False)),
    ("scaler", StandardScaler())
])

# Non-skewed numeric pipeline: Impute -> Scale
normal_num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# ColumnTransformer: different logic per column group
preprocess = ColumnTransformer(
    transformers=[
        ("skewed", skewed_num_pipe, skewed_cols),
        ("normal", normal_num_pipe, non_skewed_cols),
    ],
    remainder="drop"
)


In [11]:
# -----------------------------
# 7) Wrap into a full ML pipeline (preprocess + model)
# -----------------------------
model = Ridge(alpha=1.0, random_state=42)

full_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model)
])



In [12]:
# -----------------------------
# 8) Baseline pipeline for comparison (Impute + Scale for ALL numeric)
# -----------------------------
baseline_preprocess = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

baseline_pipeline = Pipeline(steps=[
    ("preprocess", baseline_preprocess),
    ("model", Ridge(alpha=1.0, random_state=42))
])


In [13]:
# -----------------------------
# 9) Evaluation helper
# -----------------------------
def evaluate(pipe, name: str):
    pipe.fit(X_train, y_train)

    pred_tr = pipe.predict(X_train)
    pred_te = pipe.predict(X_test)

    mae_tr = mean_absolute_error(y_train, pred_tr)
    mae_te = mean_absolute_error(y_test, pred_te)

    r2_tr = r2_score(y_train, pred_tr)
    r2_te = r2_score(y_test, pred_te)

    print(f"\n=== {name} ===")
    print(f"Train MAE: {mae_tr:,.2f} | Test MAE: {mae_te:,.2f} | Gap(Test-Train): {mae_te - mae_tr:,.2f}")
    print(f"Train R² : {r2_tr:,.4f} | Test R² : {r2_te:,.4f} | Gap(Train-Test): {r2_tr - r2_te:,.4f}")

    return pipe



In [14]:
# -----------------------------
# 10) Run + compare
# -----------------------------
baseline_pipeline = evaluate(baseline_pipeline, "Baseline: Impute+Scale (All Numeric)")
full_pipeline = evaluate(full_pipeline, "Automated: Skew-aware Pipelines (Power+Scale for skewed cols)")



=== Baseline: Impute+Scale (All Numeric) ===
Train MAE: 2,204.83 | Test MAE: 2,061.45 | Gap(Test-Train): -143.38
Train R² : 0.9748 | Test R² : 0.9908 | Gap(Train-Test): -0.0160

=== Automated: Skew-aware Pipelines (Power+Scale for skewed cols) ===
Train MAE: 16,084.76 | Test MAE: 20,336.93 | Gap(Test-Train): 4,252.16
Train R² : 0.4640 | Test R² : 0.3575 | Gap(Train-Test): 0.1065


In [15]:
# -----------------------------
# 11) Show how to reuse the same pipeline for NEW data
# -----------------------------
# Example: take 5 rows as "new incoming" data (in real life, this could be new customers)
new_data = X_test.sample(5, random_state=7)
pred_new = full_pipeline.predict(new_data)

print("\nExample predictions on new data (5 rows):")
print(pd.DataFrame({"predicted_target_sales": pred_new.round(2)}))

print("\nDone. This pipeline can now be saved and reused consistently in production.")



Example predictions on new data (5 rows):
   predicted_target_sales
0                 2730.21
1                39951.71
2                 9725.31
3               -26509.46
4                50723.50

Done. This pipeline can now be saved and reused consistently in production.
